# Instruction

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.  

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.  

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.  

Your submission will be a link to your Jupyter Notebook on your Github repository.  

In [121]:
# Import libraries

import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [122]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [123]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [124]:
# parse the HTML into BeautifulSoup parse tree format
soup = BeautifulSoup(page, 'lxml')

In [125]:
# look at the HTML code
# print(soup.prettify())

In [126]:
# find all table tag from HTML
all_tables = soup.find_all('table')

In [127]:
# find the table with the postcode
right_table = soup.find('table', class_='wikitable sortable')

Read and assign to pandas DataFrame

In [128]:
A = []
B = []
C = []

for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True).replace('\n',''))

In [129]:
df = pd.DataFrame(A,columns=['Postcode'])
df['Borought'] = B
df['Neighbourhood']=C

To create the above dataframe:  

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood  
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.  
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.  
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.  
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.  

In [130]:
df.head()

,Postcode,Borought,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove row with Not assinged Borought

In [131]:
df = df[df.Borought != 'Not assigned']

Join the duplicate Postcode and Borought rows

In [132]:
dfX = df.groupby(['Postcode','Borought']).agg(', '.join)

In [133]:
dfX = dfX.reset_index()

In [134]:
dfX.head()

,Postcode,Borought,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Replace the rows with Not assigned Neighbourhood with value in Borought

In [135]:
dfX.loc[dfX.Neighbourhood == 'Not assigned','Neighbourhood'] = dfX.Borought

In [136]:
dfX[dfX.Postcode == 'M9A']

,Postcode,Borought,Neighbourhood
93,M9A,Queen's Park,Queen's Park


In [137]:
dfX.head()

,Postcode,Borought,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [139]:
dfX.shape

(103, 3)